In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull


In [ ]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

# Set up parties in the market/ Submit liquidity provision/ Control midprice
MM_WALLET = WalletConfig("mm", "pin")

# The party to send selling/buying MOs to hit LP orders
TRADER_WALLET = WalletConfig("Zl3pLs6Xk6SwIK7Jlp2x", "bJQDDVGAhKkj3PVCc7Rr")

# The party randomly post LOs at buy/sell side to simulate real Market situation
RANDOM_WALLET = WalletConfig("OJpVLvU5fgLJbhNPdESa", "GmJTt9Gk34BHDlovB7AJ")

# The party to terminate the market and send settlment price
TERMINATE_WALLET = WalletConfig("FJMKnwfZdd48C8NqvYrG", "bY3DxwtsCstMIIZdNpKs")

wallets = [MM_WALLET, TRADER_WALLET, RANDOM_WALLET, TERMINATE_WALLET]

In [ ]:
vega = VegaServiceNull(run_with_console=True)
vega.start()

In [ ]:
for wallet in wallets:
    vega.create_wallet(wallet.name, wallet.passphrase)

In [ ]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000000000,
)

vega.forward("10s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=1e10,
)

In [ ]:
tdai_id = vega.find_asset_id(symbol="tDAI")

In [ ]:

vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=1e6
)
vega.mint(
    TERMINATE_WALLET.name,
    tdai_id,
    amount=1e2
)
vega.mint(
    RANDOM_WALLET.name,
    tdai_id,
    amount=1e6
)

In [ ]:
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
    )

In [ ]:
market_id = vega.all_markets()[0].id

In [ ]:
market_id

In [ ]:
%%timeit
vega.submit_order(
    trading_wallet=RANDOM_WALLET.name,
    market_id=market_id,
    time_in_force="TIME_IN_FORCE_GTC",
    order_type="TYPE_LIMIT",
    side="SIDE_SELL",
    volume=1,
    price=50000,
)

In [ ]:
%%timeit
vega.submit_order(
    trading_wallet=RANDOM_WALLET.name,
    market_id=market_id,
    time_in_force="TIME_IN_FORCE_GTC",
    order_type="TYPE_LIMIT",
    side="SIDE_SELL",
    volume=1,
    price=50000,
    wait=False
)

In [ ]:
%%timeit
for _ in range(10):
    vega.market_info(market_id=market_id)